In [1]:
from scenario_runner import ScenarioRunner
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.component.map.base_map import BaseMap
from metadrive.component.map.pg_map import MapGenerateMethod
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert
from pprint import pprint
from PIL import Image
import numpy as np
import logging

In [2]:
def create_env(seed, map_config) -> MetaDriveEnv:
    # ===== Fidelity Config =====
    fidelity_params = dict(
        # decision_repeat=self.decision_repeat, physics_world_step_size=self.dt
    )

    # ===== Termination Scheme =====
    termination_sceme = dict(
        out_of_route_done=False,
        on_continuous_line_done=False,
        crash_vehicle_done=False,
        crash_object_done=False,
        crash_human_done=False,
    )
    # # ===== Map Config =====
    # map_config = {
    #     BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_NUM,
    #     BaseMap.GENERATE_CONFIG: 5,
    # }

    cfg = dict(
        # use_render=True,
        log_level=logging.INFO,  # logging.DEBUG
        start_seed=seed,
        map_config=map_config,
        **termination_sceme,
        **fidelity_params,
    )
    env = MetaDriveEnv(config=cfg)
    return env

In [3]:
env = create_env(
    123,
    map_config={
        BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_NUM,
        BaseMap.GENERATE_CONFIG: 5,
    },
)

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None


In [4]:
_, reset_info = env.reset()
frames = []

[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 123, Num Scenarios : 1


In [5]:
# for i in range(500):

#     action = expert(env.agent, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     frames.append(
#         env.render(
#             mode="topdown",
#             window=False,
#             screen_record=True,
#         )
#     )

#     if terminated or truncated:
#         break

In [6]:
def get_map_img(env) -> Image:
    """Get map image of the current environment"""
    map = env.current_map.get_semantic_map(
        env.current_map.get_center_point(),
    )
    map = map.squeeze()  # reduce dimensionality
    map = (map * 255 * 4).astype(np.uint8)
    img = Image.fromarray(map)
    return img

In [7]:
# env.top_down_renderer.generate_gif(gif_name="first.gif")
# Image.fromarray(frames[-1])

In [8]:
env.agent.navigation.current_road.block_ID()

'>'

In [9]:
block_sequence = env.current_map.get_meta_data()["block_sequence"]
block_sequence

[{'id': 'I', 'pre_block_socket_index': None},
 {'radius': 10.0,
  'decrease_increase': 0,
  'change_lane_num': 0,
  'id': 'X',
  'pre_block_socket_index': '0I-socket0'},
 {'angle': 93.49340057373047,
  'radius': 43.858543395996094,
  'length': 61.55261993408203,
  'dir': 1,
  'id': 'C',
  'pre_block_socket_index': '1X-socket0'},
 {'angle': 60.0,
  'inner_radius': 17.922710418701172,
  'exit_radius': 5.974236965179443,
  'id': 'O',
  'pre_block_socket_index': '2C-socket0'},
 {'angle': 60.0,
  'inner_radius': 44.49692916870117,
  'exit_radius': 14.83230972290039,
  'id': 'O',
  'pre_block_socket_index': '3O-socket1'},
 {'radius': 10.0,
  'decrease_increase': 0,
  'change_lane_num': 0,
  'id': 'X',
  'pre_block_socket_index': '4O-socket2'}]

In [10]:
first_npcs = env.agent_manager.get_objects()
for vehicles in first_npcs.values():
    s = vehicles.get_state()
    print(s["spawn_road"], s["destination"])

('>', '>>') ('5X2_0_', '5X2_1_')
('-1X2_1_', '-1X2_0_') ('->>>', '->>')
('-1X1_1_', '-1X1_0_') ('->>>', '->>')
('1X0_1_', '2C0_0_') ('5X2_0_', '5X2_1_')
('1X0_1_', '2C0_0_') ('5X2_0_', '5X2_1_')
('1X0_1_', '2C0_0_') ('5X2_0_', '5X2_1_')
('1X0_1_', '2C0_0_') ('5X2_0_', '5X2_1_')
('-3O2_3_', '-3O2_2_') ('->>>', '->>')
('-3O0_3_', '-3O0_2_') ('->>>', '->>')
('3O0_0_', '3O0_1_') ('5X2_0_', '5X2_1_')
('-3O2_3_', '-3O2_2_') ('->>>', '->>')
('4O1_0_', '4O1_1_') ('5X2_0_', '5X2_1_')
('4O0_0_', '4O0_1_') ('5X2_0_', '5X2_1_')
('4O0_0_', '4O0_1_') ('5X2_0_', '5X2_1_')
('4O1_0_', '4O1_1_') ('5X2_0_', '5X2_1_')
('4O1_0_', '4O1_1_') ('5X2_0_', '5X2_1_')
('-5X2_1_', '-5X2_0_') ('->>>', '->>')
('-5X0_1_', '-5X0_0_') ('->>>', '->>')
('-5X0_1_', '-5X0_0_') ('->>>', '->>')


In [11]:
# get_map_img(env)

In [12]:
env.close()

In [13]:
env = create_env(
    1,
    map_config={
        BaseMap.GENERATE_TYPE: MapGenerateMethod.PG_MAP_FILE,
        BaseMap.GENERATE_CONFIG: block_sequence,
    },
)

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None


In [14]:
# get_map_img(env)

In [15]:
_, reset_info = env.reset()
frames = []

[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 1, Num Scenarios : 1


In [16]:
# for i in range(500):

#     action = expert(env.agent, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     frames.append(
#         env.render(
#             mode="topdown",
#             window=False,
#             screen_record=True,
#         )
#     )

#     if terminated or truncated:
#         break

In [17]:
# env.top_down_renderer.generate_gif(gif_name="replayed_diffrent_seed.gif")
# Image.fromarray(frames[-1])

In [18]:
env.current_map.blocks[0]

FirstPGBlock, ID:0I

In [19]:
new_npc = env.agent_manager.get_objects()
for vehicles in new_npc.values():
    s = vehicles.get_state()
    length, width, height = s["size"]
    print(f"{length = }, {width = }, {height = }")
    # print(f"{s['spawn_road'] = }, {s['destination'] = }")

length = 4.515, width = 1.852, height = 1.19
length = 5.74, width = 2.3, height = 2.8
length = 4.6, width = 1.85, height = 1.37
length = 5.74, width = 2.3, height = 2.8
length = 4.6, width = 1.85, height = 1.37
length = 4.87, width = 2.046, height = 1.85
length = 5.74, width = 2.3, height = 2.8
length = 4.87, width = 2.046, height = 1.85
length = 4.6, width = 1.85, height = 1.37
length = 4.3, width = 1.7, height = 1.7
length = 4.3, width = 1.7, height = 1.7
length = 4.87, width = 2.046, height = 1.85
length = 4.6, width = 1.85, height = 1.37
length = 5.74, width = 2.3, height = 2.8
length = 4.6, width = 1.85, height = 1.37
length = 4.87, width = 2.046, height = 1.85
length = 4.87, width = 2.046, height = 1.85
length = 4.6, width = 1.85, height = 1.37
length = 4.87, width = 2.046, height = 1.85
